In [1]:
%pylab inline
%autosave 25

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


Autosaving every 25 seconds


## Get some data

In [2]:
data_loc = '../data/FDA-COVID19_files_v1.0/'

In [3]:
store = pd.HDFStore(data_loc + 'sampled_data.h5')
df_features = pd.DataFrame(store['df' ])
store.close()
print('rows: {:,}, columns: {:,}'.format(len(df_features), len(df_features.columns)))

df_features.head()

rows: 22,172, columns: 16,391


,cid,pid,activity,AEK,VEL,EKF,LGM,VKN,LKP,NEE,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
58,204,Q99VQ4,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
97,204,EDT84149,0,3.936758,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
98,204,AAX80043,0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
126,204,P0C6X7,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,2.367871,...,0,0,0,0,0,0,0,0,0,0
136,8549,P08659,0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0


## Check column variance

How many columns have zero variance? We can drop those.

In [5]:
no_var_cols = [col for col in df_features.columns if df_features[col].nunique() == 1]

len(no_var_cols)

0

In [ ]:
var_cols = [col for col in df_features.columns if df_features[col].nunique() > 1]
df_novar_cols_dropped = df_features[var_cols].copy()

print('Dropped {:,} columns that have zero variance.'.format(len(df_features.columns)-len(var_cols)))

print('rows: {:,}, columns: {:,}'.format(len(df_novar_cols_dropped), len(df_novar_cols_dropped.columns)))

## Scale the data

In [ ]:
df_features.drop(columns=['cid', 'pid', 'activity'], inplace=True)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_features.values)

In [6]:
scaled_data.T[1000].mean()

-5.127495838066752e-18

## Find highly correlated columns

In [7]:
df_scaled = pd.DataFrame(scaled_data, columns=df_features.columns)

In [8]:
df_scaled.head()

,AEK,VEL,EKF,LGM,VKN,LKP,NEE,TPN,SRL,KEY,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
1,4.806774,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,3.380128,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
2,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
3,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,4.585527,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
4,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111


In [9]:
a = [0, 1, 2, 3, 4, 5, 6]

res = [(x, y) for i,x in enumerate(a) for j,y in enumerate(a) if i > j]

res

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5)]

In [62]:
b = [1,1,2,3,4,5,6]
np.corrcoef(a, b)[0][1]

0.9882117688026186

In [60]:
from IPython.display import display, clear_output

cols = df_scaled.columns.tolist()

corr_cols = []

def func(x, y):
    i = len(corr_cols)
    clear_output(wait=True)
    display('processed column {:,}'.format(i))
        
    corr_cols.append(x)
    return (x, y)
    

out = [func(x, y) for i,x in enumerate(cols) for j,y in enumerate(cols) if i > j and 
       np.corrcoef(df_scaled[x], df_scaled[y])[0][1] > 0.9]

'processed column 6,616'

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
len(out)

## Find correlated columns within the fingerpirnt data

In [4]:
df_fingerprints = pd.read_csv(data_loc+'drug_features/fingerprints.csv', index_col=0)

print('Number of rows: {:,}\n'.format(len(df_fingerprints)))
print('Number of columns: {:,}\n'.format(len(df_fingerprints.columns)))

Number of rows: 91,756

Number of columns: 4,096



In [9]:
num_cols = len(df_fingerprints.columns)
num_pairwise_ops = (num_cols**2-num_cols)/2.0
num_pairwise_ops

8386560.0

In [ ]:
# See which correlation method is fastest:

In [25]:
%%timeit
np.corrcoef(df_fingerprints['1947'], df_fingerprints['389'])

552 µs ± 4.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### See https://stackoverflow.com/questions/33650188/efficient-pairwise-correlation-for-two-matrices-of-features

In [5]:
fingerprint_cols = df_fingerprints.columns.tolist()
df_sampled_fingerprint = df_features[fingerprint_cols].sample(n=1000, random_state=23)

In [6]:
# Get number of rows in either A or B
N = len(df_sampled_fingerprint)

def corr_func(a, b):
    A = np.array([df_sampled_fingerprint[a].values]).T
    B = np.array([df_sampled_fingerprint[b].values]).T

    # Store columnw-wise in A and B, as they would be used at few places
    sA = A.sum(0)
    sB = B.sum(0)

    # Basically there are four parts in the formula. We would compute them one-by-one
    p1 = N*np.dot(B.T,A)
    p2 = sA*sB[:,None]
    p3 = N*((B**2).sum(0)) - (sB**2)
    p4 = N*((A**2).sum(0)) - (sA**2)

    # Finally compute Pearson Correlation Coefficient as 2D array 
    pcorr = ((p1 - p2)/np.sqrt(p4*p3[:,None]))

    # Get the element corresponding to absolute argmax along the columns 
    return pcorr[np.nanargmax(np.abs(pcorr),axis=0),np.arange(pcorr.shape[1])][0]

In [7]:
%%timeit
corr_func('1947', '389')

99.8 µs ± 841 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
# corr_func is faster than np.corrcoef

In [10]:
from IPython.display import display, clear_output

cols = df_sampled_fingerprint.columns.tolist()

corr_cols = []

def func(x, y):
    i = len(corr_cols)
    clear_output(wait=True)
    display('processed column pair {:,} out of {:,}'.format(i, num_pairwise_ops))
        
    corr_cols.append(x)
    return (x, y)
    

out = [func(x, y) for i,x in enumerate(cols) for j,y in enumerate(cols) if i > j and 
       corr_func(x, y) > 0.9]

'processed column pair 1 out of 8,386,560.0'

In [ ]:
# It's still really slow

In [12]:
out

[('1947', '389'), ('2855', '1917')]

### Let's try https://deepgraph.readthedocs.io/en/latest/tutorials/pairwise_correlations.html

In [6]:
# use only the rows from the sampled data set
# and only the columns from the fingerprints data
fingerprint_cols = df_fingerprints.columns.tolist()
df_sampled_fingerprint = df_features[fingerprint_cols]

In [6]:
df_sampled_fingerprint.shape

(22172, 4096)

In [ ]:
import os
from multiprocessing import Pool
import deepgraph as dg

In [8]:
# create observations
from numpy.random import RandomState
prng = RandomState(0)
n_features = len(df_sampled_fingerprint.columns)
n_samples = len(df_sampled_fingerprint)

# run over a sample of the rows. Can do this multiple times and take consensus.
X = df_sampled_fingerprint.sample(n=1000, random_state=23).values.T


#X = prng.randint(100, size=(n_features, n_samples)).astype(np.float64)

# uncomment the next line to compute ranked variables for Spearman's correlation coefficients
# X = X.argsort(axis=1).argsort(axis=1)

# whiten variables for fast parallel computation later on
X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)

# save in binary format
np.save(data_loc+'samples', X)
del X

In [ ]:
step_size = 1e5
n_processes = 30

# load samples as memory-map
X = np.load(data_loc+'samples.npy', mmap_mode='r')

# create node table that stores references to the mem-mapped samples
v = pd.DataFrame({'index': range(X.shape[0])})

# connector function to compute pairwise pearson correlations
def corr(index_s, index_t):
    #features_s = X[index_s]
    #features_t = X[index_t]
    #corr = np.einsum('ij,ij->i', features_s, features_t) / n_samples
    
    features_s = X[index_s].T
    features_t = X[index_t].T
    za = features_s/np.sqrt(np.einsum('ij,ij->i', features_s, features_s))[:, None]
    zb = features_t/np.sqrt(np.einsum('ij,ij->i', features_t, features_t))[:, None]
    corr = np.einsum('ik,jk', za, zb)
    
    del za
    del zb
    del features_s
    del features_t
    
    #if 1947 in index_t:
    #    print(corr.shape)
    
    return corr

# index array for parallelization
pos_array = np.array(np.linspace(0, n_features*(n_features-1)//2, n_processes), dtype=int)

# parallel computation
def create_ei(i):

    from_pos = pos_array[i]
    to_pos = pos_array[i+1]

    # initiate DeepGraph
    g = dg.DeepGraph(v)

    # create edges
    g.create_edges(connectors=corr, step_size=step_size,
                   from_pos=from_pos, to_pos=to_pos)

    # store edge table
    g.e.to_pickle((data_loc+'correlations/{}.pickle').format(str(i).zfill(3)))

# computation
if __name__ == '__main__':
    os.makedirs(data_loc+'correlations/', exist_ok=True)
    indices = np.arange(0, n_processes - 1)
    p = Pool()
    for _ in p.imap_unordered(create_ei, indices):
        pass

In [9]:
# store correlation values
files = os.listdir(data_loc+'correlations/')
files.sort()
store = pd.HDFStore(data_loc+'e.h5', mode='w')
for f in files:
    #print((data_loc+'correlations/{}').format(f))
    et = pd.read_pickle((data_loc+'correlations/{}').format(f))
    store.append('e', et, format='t', data_columns=True, index=False)
store.close()

In [10]:
# load correlation table
e = pd.read_hdf(data_loc+'e.h5')

In [11]:
len(e)

8386560

In [12]:
col1 = df_fingerprints.columns.get_loc('1947')
col2 = df_fingerprints.columns.get_loc('389')

In [13]:
e.abs().max()

corr    0.043412
dtype: float64

In [14]:
e.query('s == 389 and t == 1947')

,,corr
s,t,
389,1947,0.043412


In [15]:
df_smpled = df_sampled_fingerprint.sample(n=1000, random_state=23)

np.corrcoef(df_smpled['1947'], df_smpled['389'])[0][1]

0.9625312037778953

In [19]:
np.corrcoef(X[1947], X[389])[0][1]

0.9625312037779014

### np.einsum for calculating pearson's correlation

https://stackoverflow.com/questions/50323786/compute-correlation-in-numpy

In [18]:
ma = np.random.random((5,6)).T
mb = np.random.random((5,6)).T
 
za = ma - ma.mean(axis=1, keepdims=True)
za /= np.sqrt(np.einsum('ij,ij->i', za, za))[:, None]
zb = mb - mb.mean(axis=1, keepdims=True)
zb /= np.sqrt(np.einsum('ij,ij->i', zb, zb))[:, None]
 
cc = np.einsum('ik,jk', za, zb)

#np.allclose(cc, np.corrcoef(ma, mb)[:5, 5:])
cc.shape

(6, 6)

In [14]:
za = ma - ma.mean(axis=1, keepdims=True)
za /= np.sqrt(np.einsum('ij,ij->i', za, za))[:, None]
cc = np.einsum('ik,jk', za, za)
cc.shape

(5, 5)

In [15]:
cc2 = np.corrcoef(ma)

In [16]:
cc

array([[ 1.        , -0.05638311, -0.06116722,  0.36717897, -0.00165396],
       [-0.05638311,  1.        ,  0.85106318, -0.44946919, -0.51894837],
       [-0.06116722,  0.85106318,  1.        , -0.51878746, -0.45389857],
       [ 0.36717897, -0.44946919, -0.51878746,  1.        ,  0.63289511],
       [-0.00165396, -0.51894837, -0.45389857,  0.63289511,  1.        ]])

In [41]:
cc2

array([[ 1.        , -0.16417269,  0.3606348 , -0.73223249, -0.0259874 ],
       [-0.16417269,  1.        , -0.55337111,  0.5003826 , -0.86897447],
       [ 0.3606348 , -0.55337111,  1.        , -0.07939764,  0.52313883],
       [-0.73223249,  0.5003826 , -0.07939764,  1.        , -0.39311631],
       [-0.0259874 , -0.86897447,  0.52313883, -0.39311631,  1.        ]])

## Find correlated columns within the binding data

In [4]:
df_binding_sites = pd.read_csv(data_loc+'protein_features/binding_sites_v1.0.csv', index_col=0)

print('Number of rows: {:,}\n'.format(len(df_binding_sites)))
print('Number of columns: {:,}\n'.format(len(df_binding_sites.columns)))

Number of rows: 4,165

Number of columns: 8,481



In [7]:
num_cols = len(df_binding_sites.columns)
num_pairwise_ops = (num_cols**2-num_cols)/2.0
num_pairwise_ops

,AEK,VEL,EKF,LGM,VKN,LKP,NEE,TPN,SRL,KEY,...,FFV,MRW,YWT,AFF,LYW,*PX,SWH,TWW,FLM,RCV
0,6.340165,5.83644,8.119470,12.750716,5.893912,3.851747,4.62339,6.412804,3.758782,5.217041,...,-0.024301,-0.020249,-0.020249,-0.020249,-0.031816,-0.020249,-0.020249,-0.020249,-0.020249,-0.02506
1,5.609618,-0.20408,9.764107,-0.084641,-0.159101,-0.222109,-0.18790,-0.119188,-0.207898,-0.127372,...,-0.024301,-0.020249,-0.020249,-0.020249,-0.031816,-0.020249,-0.020249,-0.020249,-0.020249,-0.02506
2,2.926261,-0.20408,-0.150548,-0.084641,-0.159101,-0.222109,-0.18790,-0.119188,-0.207898,-0.127372,...,-0.024301,-0.020249,-0.020249,-0.020249,-0.031816,-0.020249,-0.020249,-0.020249,-0.020249,-0.02506
3,4.296038,-0.20408,-0.150548,-0.084641,-0.159101,-0.222109,-0.18790,-0.119188,-0.207898,-0.127372,...,-0.024301,-0.020249,-0.020249,-0.020249,-0.031816,-0.020249,-0.020249,-0.020249,-0.020249,-0.02506
4,3.916715,-0.20408,-0.150548,-0.084641,-0.159101,-0.222109,-0.18790,-0.119188,-0.207898,-0.127372,...,-0.024301,-0.020249,-0.020249,-0.020249,-0.031816,-0.020249,-0.020249,-0.020249,-0.020249,-0.02506


In [ ]:
from IPython.display import display, clear_output

cols = df_binding_sites.columns.tolist()

corr_cols = []

def func(x, y):
    i = len(corr_cols)
    clear_output(wait=True)
    display('processed column pair {:,} out of {:,}'.format(i, num_pairwise_ops))
        
    corr_cols.append(x)
    return (x, y)
    

out = [func(x, y) for i,x in enumerate(cols) for j,y in enumerate(cols) if i > j and 
       np.corrcoef(df_binding_sites[x], df_binding_sites[y])[0][1] > 0.9]

In [ ]:
out

In [ ]:
df_active = df_interactions[df_interactions['activity']==1]
df_active = df_active.astype({'cid': 'category'})
df_active.head()

In [ ]:
num_active = len(df_active)
total = len(df_interactions)
print('There are {:,} active entries out of {:,} interactions ({:.2%}).'.format(num_active, total, num_active/total))

In [ ]:
print('number of unique cids is {:,}.'.format(df_active['cid'].nunique()))
print('number of unique pids is {:,}.'.format(df_active['pid'].nunique()))    

In [ ]:
import seaborn as sns; sns.set(style="white", color_codes=True)

pyplot.figure(figsize=(15, 15))
df_heatmap = df_active[:100].pivot_table(values='activity', index='cid', columns='pid', aggfunc=np.sum)
sns.heatmap(df_heatmap, annot=True)
plt.show()

In [ ]:
s_cid = df_active['cid'].value_counts()
s_pid = df_active['pid'].value_counts()

print(len(s_cid[s_cid > 10]))

print(len(s_pid[s_pid > 10]))

In [ ]:
import matplotlib.pyplot as plt

#s2 = pd.Series([1,2,3,4,5,2,3,333,2,123,434,1,2,3,1,11,11,432,3,2,4,3,3,3,54,34,24,2,223,2535334,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30000, 2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
#prob = s2.value_counts(normalize=True)
#threshold = 0.02
#mask = prob > threshold
#tail_prob = prob.loc[~mask].sum()
#prob = prob.loc[mask]
#prob['other'] = tail_prob

pyplot.figure(figsize=(20, 5))
s_cid[:100].plot(kind='bar')
plt.xticks(rotation=90)
plt.show()


In [ ]:
df_inactive = df_interactions[df_interactions['activity']==0]
df_inactive = df_inactive.astype({'cid': 'category'})
df_inactive.head()